# **Book Recommendation System — Collaborative Filtering & Content-Based Approaches**

**Author:** Milos Saric [https://saricmilos.com/]  
**Date:** November 04, 2025 - November 18th, 2025  
**Dataset:** Kaggle — *Book Recommendation Dataset*  

---

In [1]:
%load_ext autoreload
%autoreload 2

from src.dataloader import load_all_csvs_from_folder
from src.preprocess_user_books_ratings import preprocess_books_ratings_users
from pathlib import Path

In [2]:
dataset_folder = Path(r"C:\Users\Milos\Desktop\ESCAPE_9-5\PYTHON\GitHub_Kaggle_Projects\what-else-should-I-read\datasets")

In [3]:
datasets = load_all_csvs_from_folder(dataset_folder)

c:\Users\Milos\Desktop\ESCAPE_9-5\PYTHON\GitHub_Kaggle_Projects\what-else-should-I-read\src\dataloader.py:63: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  datasets[csv_file.stem] = pd.read_csv(csv_file, **read_csv_kwargs)


In [4]:
merged_df = preprocess_books_ratings_users(
    datasets["Books"],
    datasets["Ratings"],
    datasets["Users"]
)

In [5]:
merged_df.shape

(1031132, 23)

In [7]:
merged_df.head()

,user_id,age,country_clean,region,city_clean,state_clean,isbn,book_rating,book_title,book_author,...,user_num_ratings,User_rating_variability,book_avg_rating,book_num_ratings,book_rating_variability,book_popularity_score,author_avg_rating,publisher_avg_rating,book_age,User_age_Group
0,2,18.0,usa,North America,other,california,0195153448,0,Classical Mythology,Mark P. O. Morford,...,1,NaN,3.500000,2,4.949747,3.845143,3.500000,3.496732,23,2
1,8,19.0,canada,North America,other,ontario,0002005018,5,Clara Callan,Richard Bruce Wright,...,17,2.867157,4.928571,14,3.970357,13.346819,4.600000,3.638298,24,2
2,8,19.0,canada,North America,other,ontario,0060973129,0,Decision in Normandy,Carlo D'Este,...,17,2.867157,5.000000,3,4.358899,6.931472,4.600000,3.708333,34,2
3,8,19.0,canada,North America,other,ontario,0374157065,0,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,...,17,2.867157,4.272727,11,4.244783,10.617328,3.916667,3.049052,26,2
4,8,19.0,canada,North America,other,ontario,0393045218,0,The Mummies of Urumchi,E. J. W. Barber,...,17,2.867157,0.000000,1,NaN,0.000000,0.000000,3.454232,26,2


# **1. Collaborative Filtering**

For **collaborative filtering (CF)**, we only need the core **user–item interactions**: `user_id`, `isbn` or `book title encoded`, and `book_rating`.  
Other Metadata like user demographics, or book averages aren’t used in standard CF models, though they can be incorporated in **hybrid approaches**.  

CF works by learning **similarity patterns**:  
- Users who rate similar books in similar ways  
- Books that are rated similarly by similar users  

Other columns (age, country, author_avg_rating, etc.) are **side information**. They are useful for:  
- Content-based recommenders  
- Hybrid models combining CF with features  
- Analytics, fairness, or deeper insights  

But for a **pure CF matrix**, only the core user–item-rating data is needed.

In [6]:
cf_df = merged_df[['user_id', 'book_title', 'book_rating']].copy()

In [7]:
min_user_ratings = 10
min_book_ratings = 10

Groups the dataframe cf_df by each user_id. Count the number of ratings each user has made (how many books they rated).Groups the dataframe by book_title. Counts how many users rated each book.